In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm

# 1. Path

In [ ]:
paths = {
    "유형": [,"kykim/electra-kor-base"],
    "극성": [,"klue/roberta-large"],
    "시제": [,"klue/roberta-large"],
    "확신성": [,"kykim/electra-kor-base"],
    
}

# 2. Data

In [3]:
!cp data/test.csv data/result.csv

In [5]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm
from datasets import load_dataset

tds = load_dataset(
    "csv",
    data_files={"predict": "data/result.csv"}
)

Using custom data configuration default-1f9e5d8c299c88b8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating predict split: 0 examples [00:00, ? examples/s]

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-1f9e5d8c299c88b8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def inference(model, tokenizer, category):
    pipe = pipeline("text-classification", model=model, tokenizer = tokenizer, device=0)
    result = []
    for out in tqdm(pipe(KeyDataset(tds["predict"], "문장"), batch_size=128)):
        result.append(out["label"])
    return result

In [ ]:
for category, (pretrained_model_name_or_path, plm) in path.items():
    tokenizer = AutoTokenizer.from_pretrained(plm)
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path)
    result = inference(model, tokenizer, category)
    tds["predict"] = tds["predict"].add_column(name=category, column=result)